In [ ]:
CLIENT_PORT=49053
CLIENT_HOST='0.0.0.0'
SERVER_URL='http://localhost:49052'

import datetime
from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
import pymongo
import os.path
from flask_cors import CORS
from collections import OrderedDict
import json
import requests
from urllib.parse import unquote,quote
import win32api
import shutil

app = Flask(__name__)
app.config['TEMPLATES_AUTO_RELOAD'] = True
CORS(app)

@app.after_request
def add_header(r):
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

@app.route('/',methods=["POST"])
def comand():
    t=request.json
    o=dict()
    o["type"]=t["type"]
    o["appID"]=t["appID"]
    o["args"]=OrderedDict(sorted(t["args"].items(), key=lambda x: x))
    o=OrderedDict(sorted(o.items(), key=lambda x: x))
    commmandInJSON=json.dumps(o,separators=(',', ':'))
    commandHash=sha256(commmandInJSON.encode("utf-8")).hexdigest()
    commandObject=dict()
    commandObject["appID"]=o["appID"]
    commandObject["type"]=o["type"]
    commandObject["commandHash"]=commandHash
    commandObject["serverSign"]=t["serverSign"]
    commandObject["appURL"]=request.headers.get("Referer")
    x = requests.post(SERVER_URL, json = commandObject)
    responseObject=dict()
    responseObject["auth"]=x.text == "True"
    if responseObject["auth"] == True:
        try:
            if t["type"]=="read":
                responseObject["result"]=dict()
                responseObject["result"]["path"]=t["args"]["path"]
                with open(unquote(t["args"]["path"]), "rb") as file:
                    responseObject["result"]["bytes"]=list(file.read())
                    responseObject["result"]["success"]="True"
            if t["type"]=="write":
                responseObject["result"]=dict()
                responseObject["result"]["path"]=t["args"]["path"]
                with open(unquote(t["args"]["path"]), "wb") as file:
                    file.write(bytearray(t["args"]["bytes"]))
                    responseObject["result"]["success"]="True"
            if t["type"]=="list":
                responseObject["result"]=dict()
                path=unquote(t["args"]["path"])
                if path=="":
                    disks = win32api.GetLogicalDriveStrings()
                    disks = disks.split('\000')[:-1]
                    responseObject["result"]["path"]=""
                    responseObject["result"]["files"]=list()
                    responseObject["result"]["folders"]=[{"path":i.replace("\\", "/"),"title":i.replace("\\", "/")} for i in disks]
                else:
                    if path[0]=="%":
                        path=os.environ[path[1:-1]].replace("\\", "/")
                    if path[-1]!="/":
                        path=path+"/"
                    directoryPath,folders,files=next(os.walk(path))
                    folders=[{"title":quote(i),
                              "modify":os.stat(os.path.join(directoryPath,i).replace("\\", "/")).st_mtime,
                              "create":os.stat(os.path.join(directoryPath,i).replace("\\", "/")).st_ctime,
                              "path":quote(os.path.join(directoryPath,i).replace("\\", "/"))} for i in folders]
                    files=[{"title":quote(i),
                            "size":os.stat(os.path.join(directoryPath,i).replace("\\", "/")).st_size,
                            "modify":os.stat(os.path.join(directoryPath,i).replace("\\", "/")).st_mtime,
                            "create":os.stat(os.path.join(directoryPath,i).replace("\\", "/")).st_ctime,
                            "path":quote(os.path.join(directoryPath,i).replace("\\", "/"))} for i in files]
                    responseObject["result"]["path"]=directoryPath
                    responseObject["result"]["files"]=files
                    responseObject["result"]["folders"]=folders
                responseObject["result"]["success"]="True"
            if t["type"]=="delete":
                responseObject["result"]=dict()
                path=unquote(t["args"]["path"]).replace("\\", "/")
                if os.path.isdir(path):
                    shutil.rmtree(path)
                else:
                    os.remove(path)
                responseObject["result"]["success"]="True"
            if t["type"]=="mkdir":
                responseObject["result"]=dict()
                path=unquote(t["args"]["path"])
                if path=="":
                    responseObject["result"]["success"]="False"
                else:
                    if path[0]=="%":
                        responseObject["result"]["success"]="False"
                    else:
                        os.mkdir(path)
                        responseObject["result"]["success"]="True"
            if t["type"]=="rename":
                responseObject["result"]=dict()
                path=unquote(t["args"]["path"])
                newName=unquote(t["args"]["newName"])
                if path=="":
                    responseObject["result"]["success"]="False"
                else:
                    if path[0]=="%":
                        responseObject["result"]["success"]="False"
                    else:
                        os.rename(path, newName)
                        responseObject["result"]["success"]="True"
        except Exception:
            responseObject["result"]["success"]="False"
    return responseObject
app.run(host=CLIENT_HOST, port=CLIENT_PORT, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:49053/ (Press CTRL+C to quit)
